In [109]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from environ import Env
from ast import literal_eval
from tqdm import tqdm
from collections import defaultdict
# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(os.curdir).resolve().parent.parent
env = Env()
env_path = BASE_DIR / "django/.env"
if env_path.exists():
    with env_path.open("rt", encoding="utf8") as f:
        env.read_env(f, overwrite=True)

dbname = env.get_value('GCPDB_NAME')
user = env.get_value('GCPDB_USER')
pw = env.get_value('GCPDB_PASSWORD')
host = env.get_value('GCPDB_HOST')

from sqlalchemy import create_engine
# echo를 true로 설정하면 command창에 실행된 sql문이 뜨게 됨
engine = create_engine(f'mysql+mysqldb://{user}:{pw}@{host}:3306/{dbname}?charset=utf8')

/opt/ml/project2/Airflow/02_Ensemble


In [112]:
Path(os.curdir).resolve().parent

PosixPath('/opt/ml/project2/Airflow')

In [110]:
BASE_DIR

PosixPath('/opt/ml/project2')

In [25]:
df = pd.read_sql_query("select * from common_batchtrain", engine)
df.head()

,id,LoginUser_id,model_name,model_path,recommended_movie_list,create_time
0,26,1,CDAE,/opt/ml/project2/BaseLine/saved/CDAE-Feb-08-20...,"[1682, 63082, 72998, 356, 318, 2324, 79132, 48...",2023-02-08 15:39:57.337129
1,27,2,CDAE,/opt/ml/project2/BaseLine/saved/CDAE-Feb-08-20...,"[63082, 4226, 2959, 79132, 293, 6377, 99114, 3...",2023-02-08 15:39:57.337129
2,28,3,CDAE,/opt/ml/project2/BaseLine/saved/CDAE-Feb-08-20...,"[4226, 68237, 99114, 4886, 63082, 48780, 4979,...",2023-02-08 15:39:57.337129
3,29,4,CDAE,/opt/ml/project2/BaseLine/saved/CDAE-Feb-08-20...,"[2997, 2959, 5445, 3535, 4995, 4878, 4571, 598...",2023-02-08 15:39:57.337129
4,30,5,CDAE,/opt/ml/project2/BaseLine/saved/CDAE-Feb-08-20...,"[2959, 48780, 318, 56367, 4886, 4226, 72998, 4...",2023-02-08 15:39:57.337129


In [16]:
weight_dict={"ADMMSLIM":0.7, "EASE":1, "RecVAE":0.5}

In [79]:
dataframe_list = []

print('순위별 가중치값 입력(ex: 1 0.9 0.8 ...)')
rank_ratio = np.linspace(1,0,501)[:500].tolist()
rank_len = len(rank_ratio)

print(f"앙상블 모델 개수: {len(weight_dict)}")

순위별 가중치값 입력(ex: 1 0.9 0.8 ...)
앙상블 모델 개수: 3


In [89]:
## 앙상블할 추천 결과 df 리스트
dataframe_list = []
## 모델 가중치
ratios = []
for model_name in weight_dict:
    ratios.append(weight_dict[model_name])
    df_model = df[df.model_name==model_name].copy()
    last_date = df_model.create_time.max()
    print(f"{model_name} 모델이 추천한 마지막 시간은 {last_date.strftime('%Y년 %m월 %d일 %H시 %M분')}")

    df_model_last = df_model[df_model.create_time==last_date].copy()
    print(f"그 때 추천한 유저는 {df_model_last.shape[0]}명")
    df_model_last.loc[:,'recommended_movie_list'] = df_model_last.recommended_movie_list.map(literal_eval)
    df_model_last = df_model_last[['LoginUser_id','recommended_movie_list']].explode('recommended_movie_list').copy()
    dataframe_list.append(df_model_last)
print(f"가중치 {weight_dict}")

ADMMSLIM 모델이 추천한 마지막 시간은 2023년 02월 12일 02시 39분
그 때 추천한 유저는 11명
EASE 모델이 추천한 마지막 시간은 2023년 02월 09일 01시 25분
그 때 추천한 유저는 11명
RecVAE 모델이 추천한 마지막 시간은 2023년 02월 08일 19시 59분
그 때 추천한 유저는 11명
가중치 {'ADMMSLIM': 0.7, 'EASE': 1, 'RecVAE': 0.5}


In [90]:
dataframe_len = len(dataframe_list)
dataframe_len

3

In [91]:
user_list = dataframe_list[0]['LoginUser_id'].unique()
user_list

array([ 1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 13])

In [103]:
K=500
result = []
tbar = tqdm(user_list, desc='Ensemble')
for user in tbar:
    temp = defaultdict(float)
    for df_idx in range(dataframe_len):
        items = dataframe_list[df_idx][dataframe_list[df_idx]['LoginUser_id'] == user]['recommended_movie_list'].values
        max_rank = min(len(items), rank_len)
        for rank_idx in range(max_rank):
            temp[items[rank_idx]] += rank_ratio[rank_idx] * ratios[df_idx]

    for key, _ in sorted(temp.items(), key=lambda x: x[1], reverse=True)[:K]:
        result.append((user, key))

Ensemble: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 366.55it/s]


In [104]:
sub = pd.DataFrame(result, columns=['LoginUser_id', 'recommended_movie_list'])

In [105]:
# DB에 쓸 결과 파일 생성
result = sub.groupby('LoginUser_id').recommended_movie_list.apply(list).reset_index()

# 결과 파일 정제
result['model_name'] = "ADER"
result['model_path'] = "No Path"
result['create_time'] = str(pd.Timestamp.now())
result['id'] = 0

In [106]:
# 컬럼 순서 맞추기
cols = ['id','LoginUser_id', 'model_name','model_path', 'recommended_movie_list', 'create_time']
result = result[cols]

In [108]:
len(result.loc[0,'recommended_movie_list'])

500

In [101]:
# DB에 쓰기
result.astype(str).to_sql(name='common_batchtrain', con=engine, index=False, if_exists='append')

11